In [1]:
import numpy
from collections import deque
import pandas
import math

In [2]:
def iterate_fund(ladder, yield_curve, max_maturity):
    reduce_maturity(ladder)
    
    payments = get_payments(ladder)

    sold_bond = ladder.popleft()
    payments += sold_bond.value(yield_curve)

    new_bond = Bond(payments, yield_curve[max_maturity-1], max_maturity)
    ladder.append(new_bond)
    
    # This happens *after* we sell the shortest bond and buy a new long one
    # (at least, that's what longinvest does...)
    nav = get_nav(ladder, yield_curve)

    return (ladder, payments, nav)

def get_nav(ladder, rates):
    return sum((b.value(rates) for b in ladder))

def get_payments(ladder):
    return sum((b.gen_payment() for b in ladder))

def reduce_maturity(ladder):
    for b in ladder:
        b.maturity -= 1
    return ladder

In [3]:
class Bond:
    def __init__(self, face_value, yield_pct, maturity):
        self.face_value = face_value
        self.yield_pct = yield_pct
        self.maturity = maturity
        
    def __repr__(self):
        return ('Maturity: %d | Yield: %.2f%% | Face Value: $%.2f' % (self.maturity, self.yield_pct * 100, self.face_value))
        
    def gen_payment(self):
        return self.face_value * self.yield_pct
    
    def value(self, rates):
        value = numpy.pv(rates[self.maturity - 1], self.maturity, self.gen_payment(), self.face_value)
        return -value

In [4]:
def bootstrap(yield_curve, max_bonds, min_maturity):
    bond_yield = yield_curve[max_bonds - 1]
    ladder = deque()
    starting_face_value = 50 # chosen arbitrarily (to match longinvest)

    for i, j in zip(range(max_bonds), range(min_maturity, max_bonds+1)):
        face_value = pow(1 + bond_yield, i) * starting_face_value
        b = Bond(face_value, bond_yield, j)
        ladder.append(b)
    return ladder
bootstrap([.0532]*10, 10, 2)

deque([Maturity: 2 | Yield: 5.32% | Face Value: $50.00,
       Maturity: 3 | Yield: 5.32% | Face Value: $52.66,
       Maturity: 4 | Yield: 5.32% | Face Value: $55.46,
       Maturity: 5 | Yield: 5.32% | Face Value: $58.41,
       Maturity: 6 | Yield: 5.32% | Face Value: $61.52,
       Maturity: 7 | Yield: 5.32% | Face Value: $64.79,
       Maturity: 8 | Yield: 5.32% | Face Value: $68.24,
       Maturity: 9 | Yield: 5.32% | Face Value: $71.87,
       Maturity: 10 | Yield: 5.32% | Face Value: $75.69])

In [5]:
BOND_RATES = pandas.read_csv('oecd_bond_rates.csv', index_col=0)
BOND_RATES.head()

,AUS,AUT,BEL,CAN,DNK,FRA,DEU,ITA,JPN,NLD,...,SWE,CHE,GBR,USA,ALL AVERAGE,STD-DEV,16 COUNTRIES AVERAGE,STD-DEV.1,NO JPN CHE AVERAGE,STD-DEV.2
YEAR,,,,,,,,,,,,,,,,,,,,,
1970,0.0665,0.0781,0.0771,0.0791,0.1057,0.0812,0.0830,0.0901,0.0719,0.0822,...,0.0739,0.0582,0.0922,0.0735,0.0784,0.0119,0.0784,0.0119,0.0783,0.0085
1971,0.0671,0.0771,0.0729,0.0695,0.1067,0.0779,0.0798,0.0834,0.0728,0.0735,...,0.0723,0.0527,0.0890,0.0616,0.0747,0.0126,0.0747,0.0126,0.0740,0.0080
1972,0.0583,0.0736,0.0696,0.0723,0.1037,0.0742,0.0786,0.0747,0.0669,0.0688,...,0.0729,0.0497,0.0890,0.0621,0.0718,0.0127,0.0718,0.0127,0.0714,0.0082
1973,0.0693,0.0825,0.0735,0.0756,0.1107,0.0833,0.0931,0.0742,0.0726,0.0792,...,0.0738,0.0560,0.1071,0.0684,0.0788,0.0151,0.0788,0.0151,0.0785,0.0121
1974,0.0904,0.0974,0.0857,0.0890,0.1455,0.1056,0.1037,0.0987,0.0926,0.0982,...,0.0778,0.0715,0.1477,0.0756,0.0967,0.0231,0.0967,0.0231,0.0951,0.0200


In [6]:
BILL_RATES = pandas.read_csv('oecd_bill_rates.csv', index_col=0)
BILL_RATES.head()

,AUS,AUT,BEL,CAN,CHE,CHL,COL,CRI,CZE,DEU,...,SVN,SWE,USA,ZAF,ALL AVERAGE,STD-DEV,16 COUNTRIES AVERAGE,STD-DEV.1,NO JPN CHE AVERAGE,STD-DEV.2
YEAR,,,,,,,,,,,,,,,,,,,,,
1970,0.07108,NaN,0.07808,0.07454,NaN,NaN,NaN,NaN,NaN,0.09407,...,NaN,0.084375,0.07564,NaN,0.08044,0.00910,0.08044,0.00910,0.08044,0.00910
1971,0.07008,NaN,0.05004,0.04569,NaN,NaN,NaN,NaN,NaN,0.07148,...,NaN,0.058542,0.05005,NaN,0.05837,0.01121,0.05837,0.01121,0.05837,0.01121
1972,0.05083,NaN,0.03829,0.05098,NaN,NaN,NaN,NaN,NaN,0.05611,...,NaN,0.037500,0.04666,NaN,0.04966,0.00651,0.04966,0.00651,0.04966,0.00651
1973,0.06983,NaN,0.06288,0.07301,NaN,NaN,NaN,NaN,NaN,0.12143,...,NaN,0.027917,0.08416,NaN,0.08377,0.02108,0.08377,0.02108,0.08377,0.02108
1974,0.13158,NaN,0.10296,0.10554,0.10165,NaN,NaN,NaN,NaN,0.09903,...,NaN,0.068542,0.10244,NaN,0.10967,0.01323,0.11049,0.01407,0.11196,0.01482


In [7]:
def build_yield_curve(bill_rate, bond_rate, yield_curve_size=30):
    s = pandas.Series(math.nan, index=numpy.arange(yield_curve_size))
    s.iloc[0] = bill_rate
    s.iloc[yield_curve_size-1] = bond_rate
    s.interpolate(inplace=True)
    s.fillna(method='backfill', inplace=True)    

    return s.tolist()

In [8]:
def get_rate_pair_at(year, country):
    bond_rate = BOND_RATES.loc[year][country]
    bill_rate = BILL_RATES.loc[year][country]
    return (bill_rate, bond_rate)

['%.2f' % (s*100) for s in build_yield_curve(*get_rate_pair_at(1970, 'AUS'))]

['7.11',
 '7.09',
 '7.08',
 '7.06',
 '7.04',
 '7.03',
 '7.01',
 '7.00',
 '6.98',
 '6.97',
 '6.95',
 '6.93',
 '6.92',
 '6.90',
 '6.89',
 '6.87',
 '6.86',
 '6.84',
 '6.82',
 '6.81',
 '6.79',
 '6.78',
 '6.76',
 '6.74',
 '6.73',
 '6.71',
 '6.70',
 '6.68',
 '6.67',
 '6.65']

In [9]:
yield_curve = build_yield_curve(*get_rate_pair_at(1970, 'AUS'))
bootstrap(yield_curve, 10, 4)

deque([Maturity: 4 | Yield: 6.97% | Face Value: $50.00,
       Maturity: 5 | Yield: 6.97% | Face Value: $53.48,
       Maturity: 6 | Yield: 6.97% | Face Value: $57.21,
       Maturity: 7 | Yield: 6.97% | Face Value: $61.19,
       Maturity: 8 | Yield: 6.97% | Face Value: $65.46,
       Maturity: 9 | Yield: 6.97% | Face Value: $70.02,
       Maturity: 10 | Yield: 6.97% | Face Value: $74.89])

In [10]:
def loop(ladder, rates, max_maturity, start_year, end_year):
    df = pandas.DataFrame(columns=['NAV', 'Payments', 'Change'], index=numpy.arange(start_year, end_year + 1))
    
    for year in range(start_year, end_year+1):
        c = rates.loc[year]
        (ladder, payments, nav) = iterate_fund(ladder, build_yield_curve(c['bills'], c['bonds']), max_maturity)
        df.loc[year] = {'NAV' : nav, 'Payments' : payments}

    calculate_returns(df)
    return df

def calculate_returns(df):
    # Longinvest calculates the return based on comparison's to
    # next year's NAV. So I'll do the same. Even though that seems
    # weird to me. Maybe it's because the rates are based on January?
    # Hmmm...that sounds plausible.
    max_row = df.shape[0]

    for i in range(max_row - 1):
        next_nav = df.iloc[i+1]['NAV']
        nav = df.iloc[i]['NAV']
        change = (next_nav - nav) / nav
        df.iloc[i]['Change'] = change
    return df

def simulate(max_maturity, min_maturity, country):
    """ This is just something to save on typing...and make clearer what the bounds on the fund are """
    # find the first non-NaN number in rates
    bonds = BOND_RATES[country].dropna()
    bills = BILL_RATES[country].dropna()
    
    start_year = 1970 #max(bills.head(1).index[0], bonds.head(1).index[0])
    if country == 'ESP': start_year = 1979

    end_year = 2017 #min(bills.tail(1).index[0], bonds.tail(1).index[0])
    
    rates = pandas.DataFrame.from_dict({'bills' : bills, 'bonds' : bonds})
    
    starting_rates = rates.loc[start_year]
    
    ladder = bootstrap(build_yield_curve(starting_rates['bills'], starting_rates['bonds']), max_maturity, min_maturity)
    return loop(ladder, rates, max_maturity, start_year, end_year)

In [26]:
simulate(30, 30, 'DNK').head()

,NAV,Payments,Change
1970,55.285,55.285,0.0968233
1971,60.6379,60.6379,0.133975
1972,68.7618,68.7618,0.0434766
1973,71.7514,71.7514,-0.123821
1974,62.8671,62.8671,0.25307


In [21]:
def t():
    country = 'USA'
    bonds = BOND_RATES[country].dropna()
    bills = BILL_RATES[country].dropna()
    rates = pandas.DataFrame.from_dict({'bills' : bills, 'bonds' : bonds})
    starting_rates = rates.loc[1970]
    yc = build_yield_curve(starting_rates['bills'], starting_rates['bonds'])
    ladder = bootstrap(build_yield_curve(starting_rates['bills'], starting_rates['bonds']), 10, 5)
    return yc, ladder

# Saving to CSV
To do anything useful, you probably want to save the results to a CSV file. Here's a commented-out example of how to do that.

In [27]:
countries = [
    'AUS',
    'AUT',
    'BEL',
    'CAN',
    'DNK',
    'FRA',
    'DEU',
    'ITA',
    'JPN',
    'NLD',
    'NOR',
#    'SGD', # 1999 onward
    'ESP', # 1979 onward
    'SWE',
    'CHE',
    'GBR',
    'USA',
    'ALL AVERAGE',
    '16 COUNTRIES AVERAGE',
    'NO JPN CHE AVERAGE',
]

pd = pandas.DataFrame(columns=countries)

for c in countries:
    print('Simulating ...', c)
    returns = simulate(30, 30, c)
    pd[c] = returns['Change']

pd.head()
pd.to_csv('oecd_30_returns.csv')

Simulating ... AUS
Simulating ... AUT
Simulating ... BEL
Simulating ... CAN
Simulating ... DNK
Simulating ... FRA
Simulating ... DEU
Simulating ... ITA
Simulating ... JPN
Simulating ... NLD
Simulating ... NOR
Simulating ... ESP
Simulating ... SWE
Simulating ... CHE
Simulating ... GBR
Simulating ... USA
Simulating ... ALL AVERAGE
Simulating ... 16 COUNTRIES AVERAGE
Simulating ... NO JPN CHE AVERAGE
